### Preprocessing

In [1]:
import os
os.chdir("..")

In [2]:
import pandas as pd
import numpy as np
import importlib, glob, pickle, torch, logging
from datetime import datetime
from concurrent import futures
from tqdm import tqdm

from src.data.insert_ids import insert_ids_in_folder
from src.data.rename_logos import rename_logos
from src.data.get_svg_meta_data import get_svg_meta_data
from src.data.generate_random_animations import random_animation_vector
from src.data.expand_viewbox import expand_viewbox_in_folder

from src.preprocessing.sort_paths import sort_by_relevance
from src.preprocessing.decompose_logo import decompose_logos_in_folder
from src.preprocessing.create_svg_embedding import _load_model, apply_embedding_model_to_svgs, combine_embeddings, encode_svg, decode_z
from src.preprocessing.deepsvg.svglib.svg import SVG
from src.preprocessing.deepsvg import utils

from src.features.get_style_attributes import *

from src.models_output.insert_animation import create_animated_svg
from src.models_output.transform_animation_predictor_output import transform_animation_predictor_output

#### 1 Rename and insert IDs in SVGs
Logos are renamed and an ID called animation_id is inserted for every path in a SVG.

In [8]:
expand_viewbox_in_folder('data/initial_svgs', 50, "data/svgs_expanded")

In [3]:
rename_logos("data/truncated_svgs_new", "data/truncated_svgs_new_renamed")

In [4]:
insert_ids_in_folder("data/svgs_expanded_renamed", "data/svgs")

#### 2 Decompose SVGs
Decompose SVGs by saving every path in a seperate SVG file. Decomposed SVGs are save to data/decomposed_svgs.

In [4]:
decompose_logos_in_folder("data/truncated_svgs_new")

#### 3 Simplify SVGs
Creates simplified SVGs and its tensors on SVG and path level  
###### UPDATE: NOT NEEDED ANYMORE

In [6]:
#preprocess_folder(data_folder="data/svgs")

In [7]:
#preprocess_folder(data_folder="data/decomposed_svgs")

#### 4 Get SVGs Metadata

In [3]:
pd.set_option('display.max_rows', None)

In [3]:
#meta_data = get_svg_meta_data("data/truncated_svgs")
#meta_data

In [5]:
meta_data.to_csv("data/meta_data/truncated_svgs_meta.csv", index=False)

In [5]:
output = open("data/meta_data/svgs_metadata.csv", 'wb')
pickle.dump(meta_data, output)
output.close()

#### 5 Get Path Embeddings
Apply hierarchical generative network to produce latent vector z

In [8]:
model = "models/hierarchical_ordered.pth.tar"

In [1]:
#apply_embedding_model_to_svgs(data_folder="data/decomposed_svgs", split_paths=True, save=True)

In [10]:
with open('data/embeddings/hierarchical_ordered_decomposed_svgs_embedding.pkl', 'rb') as f:
    path_embedding = pickle.load(f)

#### 6 Get Truncated SVG Embedding

In [8]:
apply_embedding_model_to_svgs(data_folder="data/truncated_svgs", split_paths=False, save=True)

100%|████████████████████████████████████████████████████████████████████████████████| 192/192 [00:15<00:00, 12.29it/s]


,filename,0,1,2,3,4,5,6,7,8,...,246,247,248,249,250,251,252,253,254,255
0,logo_0,-0.139803,0.517301,-0.216177,-0.224378,0.245434,1.295424,-0.751694,0.979410,-0.085270,...,-0.219360,-1.537408,-0.509325,0.224033,1.331241,0.604090,-0.434301,-0.310489,-0.737527,-0.894298
1,logo_10,0.014481,0.991770,0.440586,-0.211320,-0.179296,0.363853,-0.269891,1.362678,-0.695728,...,-0.587773,-2.270502,-0.187884,0.367311,1.406683,0.218315,0.082198,-0.028952,0.252413,-0.322137
2,logo_1,0.382652,1.325662,1.311528,-0.064719,-0.782713,0.917144,-0.723518,0.891246,-1.482842,...,-0.425215,-3.057231,-0.617269,0.342195,1.086993,0.286118,-0.092350,-0.987957,0.148480,-0.150186
3,logo_100,0.001948,0.060774,0.817412,-1.175098,0.517973,-1.232312,-0.623534,1.221800,0.616293,...,-1.644043,-2.767421,-0.083972,0.314031,0.924318,0.017703,0.671873,-1.140841,0.015591,0.465218
4,logo_101,0.322849,-0.782018,0.222675,0.245687,-0.222568,1.760160,-1.393791,1.284999,0.067373,...,-1.133171,-2.162712,-0.309598,1.302865,2.054115,-1.038654,0.044368,-0.353557,-0.675948,-0.673563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,logo_94,0.554916,0.966286,0.399162,0.090689,0.247119,-0.895295,-0.536079,0.829561,-0.274277,...,-0.235139,-1.654074,0.445131,-0.942813,0.501560,-0.255918,0.526573,-0.496769,-0.137688,0.594756
188,logo_96,1.352827,1.692985,0.016531,0.428921,-0.567170,0.695044,1.154331,0.877085,-1.598299,...,1.062907,-1.508039,1.016259,0.003275,1.068140,-0.095732,0.436575,0.260033,0.244037,-0.288786
189,logo_95,0.332031,0.927773,0.001991,0.215348,0.127213,-1.141315,-0.564558,1.094241,-0.433303,...,0.544663,-1.499968,0.184478,-1.318667,0.346078,-0.583003,0.230270,-0.692683,-0.386916,0.355028
190,logo_98,0.625203,0.970013,0.349928,0.179380,-0.471530,1.241775,0.090893,0.238759,-0.604534,...,0.321055,-0.989886,0.801984,0.035731,0.911039,-0.243607,0.503264,-0.058393,-0.635538,-0.226448


In [9]:
with open('data/embeddings/truncated_svg_embedding.pkl', 'rb') as f:
    truncated_svg_embedding = pickle.load(f)

In [10]:
truncated_svg_embedding["logo_id"] = truncated_svg_embedding["filename"].apply(lambda l: int(l.split("_")[-1]) )

In [11]:
truncated_svg_embedding.sort_values(by=['logo_id'])

,filename,0,1,2,3,4,5,6,7,8,...,247,248,249,250,251,252,253,254,255,logo_id
0,logo_0,-0.319286,0.768348,-0.230691,0.507367,-0.075261,1.508813,-0.912999,0.675434,-0.001406,...,-1.237188,-0.534437,-0.611804,1.196366,0.697434,-0.608996,-0.137433,-1.429938,-0.784427,0
2,logo_1,0.336720,1.501473,1.169257,0.030017,-0.814627,1.051143,-0.441092,0.991913,-1.523129,...,-2.788134,-0.738329,0.469090,1.012248,0.454091,-0.068043,-1.105730,0.461207,-0.313481,1
96,logo_4,1.026601,0.503723,0.393026,-0.463900,0.018286,0.778132,0.000968,0.523557,-1.192480,...,-2.094021,0.937697,1.070840,0.847270,0.806781,0.498334,-0.152665,-0.135112,-0.326979,4
107,logo_5,0.618347,1.313700,0.918294,-0.425941,-0.146768,0.430400,0.306780,2.013381,-1.111060,...,-2.927367,-1.322501,-0.160848,1.673383,0.397928,-0.588335,-0.169742,0.305947,-0.501522,5
116,logo_6,0.640700,1.018042,0.267838,-0.324064,-0.437142,0.850384,0.643671,1.262248,-0.953211,...,-0.923869,-0.737705,0.297321,1.181116,-0.196109,-0.455151,0.263948,0.172333,-0.349800,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,logo_187,1.539024,-0.040631,0.393989,0.319893,-1.290616,0.669674,-0.577846,-0.315150,-1.112488,...,-1.373711,0.187085,0.559191,0.778073,-0.717797,0.360962,-0.202559,-0.279704,0.133265,187
73,logo_188,-0.113772,1.393692,-0.200677,1.206145,-0.995082,1.400655,-0.105506,1.488463,-0.449894,...,-0.163678,-1.674516,0.243883,1.770431,-0.551852,-0.358444,-0.326204,0.229872,-0.020805,188
75,logo_189,0.398498,0.701880,0.372359,-0.059786,-0.335600,0.325044,-0.882167,1.376473,-0.090127,...,-0.969852,0.658988,0.247969,1.414171,-0.646614,0.383088,-0.130043,-0.201297,0.442788,189
76,logo_190,0.985857,-0.084097,0.303740,-0.814396,-0.259730,0.269703,-0.343166,1.259054,-0.882186,...,-2.414949,0.998506,1.311748,1.195492,-0.565383,0.687683,-0.570188,0.339194,-0.100368,190


In [37]:
#file = "data/truncated_svgs/logo_1.svg"
#z = encode_svg(filename = file, data_folder="data/truncated_svgs", split_paths=False)
#decode_z(z)

In [ ]:
with open('../../data/animated_svgs_dataframes/1646_animation_vectors.pkl', 'rb') as f:
        animation_df = pickle.load(f)

#### 7 Get Style Attributes
Get local as well as global style attributes of SVGs.

In [7]:
df_local = get_local_style_attributes("data/svgs")
df_global = get_global_style_attributes("data/svgs")

In [8]:
df_all_style_attributes = combine_style_attributes(df_global, df_local)
df_all_style_attributes

,file,animation_id,class_,fill,stroke,stroke_width,opacity,stroke_opacity
0,L_7sins_01.svg,0,st0,#FABB00,#000000,0,1.0,1.0
1,L_7sins_01.svg,1,st1,#C5007B,#000000,0,1.0,1.0
2,L_7sins_01.svg,2,st2,#EC7404,#000000,0,1.0,1.0
3,L_7sins_01.svg,3,st1,#C5007B,#000000,0,1.0,1.0
4,L_7sins_01.svg,4,st2,#EC7404,#000000,0,1.0,1.0
...,...,...,...,...,...,...,...,...
5095,L_Weidenbachhof_9_2.svg,22,st3,#FFFFFF,#000000,0,1.0,1.0
5096,L_Weidenbachhof_9_2.svg,23,st3,#FFFFFF,#000000,0,1.0,1.0
5097,L_Weidenbachhof_9_2.svg,24,st3,#FFFFFF,#000000,0,1.0,1.0
5098,L_Weidenbachhof_9_2.svg,25,st3,#FFFFFF,#000000,0,1.0,1.0


In [ ]:
#df_all_style_attributes.to_csv("data/style_attributes.csv")

In [ ]:
#output = open("data/style_attributes.pkl", 'wb')
#pickle.dump(df_all_style_attributes, output)
#output.close()

#### 8 Insert Animations

In [5]:
file = "data/svgs/L_7sins_02.svg"
animation_id = sort_by_relevance("data/path_selection/L_7sins_02")

In [6]:
for s in range(200):
    model_output = random_animation_vector(8, seed=s)
    create_animated_svg(file, animation_id, model_output, str(s))